In [ ]:
import numpy as np
import pandas as pd
from resolve_path import ajuste_path
from unidecode import unidecode

In [ ]:
pathInput = "data/input/"
pathInput = ajuste_path(pathInput)

df = pd.read_csv(pathInput + "BD_Acidentes_13_08.csv",
                 sep="@", encoding="utf-8")
print(df.info())

## Pré processamento das colunas:

In [ ]:
df.columns = df.columns.str.lower()
df.columns = df.columns.map(lambda x: unidecode(str(x)))
df.info()

## Pré processamento das linhas

### Utilitários

In [ ]:
replace_dict = {
    "na": np.nan,
    "nan": np.nan,
    "n.a": np.nan,
    "n.a.": np.nan,
    "n/a": np.nan,
    "-": np.nan,
}

### Empregado

In [ ]:
df["empregado"] = df["empregado"].str.lower()
df["empregado"] = df["empregado"].map(lambda x: unidecode(str(x)))
df["empregado"].unique()

### Classificação

In [ ]:
df["classificacao"] = df["classificacao"].str.lower()
df["classificacao"] = df["classificacao"].map(lambda x: unidecode(str(x)))
df["classificacao"].unique()

### Empresa

In [ ]:
df["empresa"] = df["empresa"].str.lower()
df["empresa"] = df["empresa"].map(lambda x: unidecode(str(x)))
df["empresa"].unique()

### Fornecedor

In [ ]:
df["fornecedor"] = df["fornecedor"].str.lower()
df["fornecedor"] = df["fornecedor"].map(lambda x: unidecode(str(x)))

df["fornecedor"] = df["fornecedor"].replace(replace_dict)
df["fornecedor"].unique()

### Colocando próprios com fornecedor como terceiro

In [ ]:
df.loc[df["fornecedor"].notna(), "empregado"] = "terceiro"
print(df["empregado"].unique())

### ID

In [ ]:
df["id"] = df["id"].str.lower()
df["id"] = df["id"].map(lambda x: unidecode(str(x)))

df["id"] = df["id"].str.lstrip("\t")

df["id"] = df["id"].replace(replace_dict)

df_eletrosul = df.loc[df["empresa"] == "cgt eletrosul"]
df_proprio = df_eletrosul.loc[df_eletrosul["empregado"] == "proprio"]
df_proprio_notna = df_proprio.dropna(subset=["id"])

print(df_proprio_notna["id"].nunique())
print(df_proprio_notna["id"].unique())

# ! voltar e resolver os dois IDs na mesma linha

### Dia

In [ ]:
df["dia"].unique()

### Mes

In [ ]:
df["mes"].unique()

### Ano

In [ ]:
df["ano"].unique()

### Descrição

In [ ]:
df["descricao"].str.lower()
df["descricao"].map(lambda x: unidecode(str(x)))

df["descricao"] = df["descricao"].replace(replace_dict)

### Coordenadas

In [ ]:
def processa_coordenada(value):
    if isinstance(value, str):
        value = value.replace(",", ".")  # Substitui vírgula por ponto
        value = value.replace("°", "")  # Remove "°"
        return float(value)

    return value


def corrige_coordenada(df):
    # Corrigir longitude digitada errada (número muito grande)
    df["longitude"] = df["longitude"].apply(
        lambda x: x / 100000 if pd.notna(x) and x < -100 else x
    )
    # Corrigir sinal das coordenadas trocado
    df["longitude"] = df["longitude"].apply(
        lambda x: x * (-1) if pd.notna(x) and x > 0 else x
    )
    # Corrigir vírgulas das coordenadas digitadas errado
    df["longitude"] = df["longitude"].apply(
        lambda x: x * 10 if pd.notna(x) and x > -10 else x
    )
    # Desconsiderar pontos na Antártica
    df = df[df["latitude"].apply(lambda x: pd.notna(
        x) and x > -40 if pd.notna(x) else True)]

    return df

Latitude

In [ ]:
print(df[["latitude", "longitude"]].isna().sum())

print(df["latitude"].unique())
df["latitude"] = df["latitude"].apply(processa_coordenada)
print(df["latitude"].unique())

Longitude

In [ ]:
print(df["longitude"].unique())
df["longitude"] = df["longitude"].apply(processa_coordenada)
print(df["longitude"].unique())

In [ ]:
df = corrige_coordenada(df)
print(df[["latitude", "longitude"]].isna().sum())

### Exportar o dataset

In [ ]:
df = df.drop(columns=["unnamed: 0"])

pathUtil = "data/util/"
pathUtil = ajuste_path(pathUtil)

df.to_csv(pathUtil + "acidentes/" + "acidentes_preprocessado.csv",
          sep="@", encoding="utf-8", index=False)